## Week 3 Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np 
import pandas as pd
import requests
import json as js
import csv

#### 1. Get the data from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data1 =pd.read_html(url)[0]
data1.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### 2. Rename columns

In [4]:
data2 = pd.DataFrame(data1)
data2.columns = ['PostalCode', 'Borough','Neighborhood']
data2.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [17]:
data2.count()

PostalCode      180
Borough         180
Neighborhood    103
dtype: int64

#### 3. Drop data with a not assigned borough

In [18]:
data3 = pd.DataFrame(data2)
data3.drop(data3[data3['Borough']=='Not assigned'].index, inplace=True)
data3.reset_index(drop=True,inplace=True)
data3.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [19]:
data3.count()

PostalCode      103
Borough         103
Neighborhood    103
dtype: int64

#### 4. Combine rows with same PostalCode

In [38]:
data4 = pd.DataFrame(data3)
data4 = data4.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x:', '.join(x)).to_frame().reset_index()
data4.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [41]:
data4.shape

(103, 3)